In [ ]:
import sys
sys.path.append('../../code')
import deepxde as dde
import matplotlib.pyplot as plt
import numpy as np
# Import tf if using backend tensorflow.compat.v1 or tensorflow
from deepxde.backend import tf

## MAIN

In [ ]:
pi = np.pi
npart = 4

def pde(x, y):
    dy_xx = dde.grad.hessian(y, x)
    return -dy_xx - (np.pi) ** 2 * tf.sin(np.pi * x)

def boundary(x, on_boundary, npart=npart):
    return on_boundary

def func(x):
    return np.sin(np.pi * x)
    
geom = dde.geometry.Interval(-1, 1)
bc = dde.icbc.DirichletBC(geom, func, boundary, npart=npart)
data = dde.data.PDE(geom, pde, bc, 26, 2, solution=func, num_test=500)

def psi_a(x):
    arr = np.ones((x.shape[0],1), dtype='float32')
    for i in range(x.shape[0]):
        if x[i] < -5/4 or x[i] > 5/4:
            arr[i] = 0
        elif x[i] >= 3/4 and x[i] <= 5/4:
            arr[i] = (1 - np.sin(2*pi*x[i]))/2
    return arr

def psi_b(x):
    arr = np.ones((x.shape[0],1), dtype='float32')
    for i in range(x.shape[0]):
        if x[i] < -5/4 or x[i] > 5/4:
            arr[i] = 0
        elif x[i] >= -5/4 and x[i] <= -3/4:
            arr[i] = (1 + np.sin(2*pi*x[i]))/2
        elif x[i] >= 3/4 and x[i] <= 5/4:
            arr[i] = (1 - np.sin(2*pi*x[i]))/2
    
    return arr

def psi_c(x):
    arr = np.ones((x.shape[0],1), dtype='float32')
    for i in range(x.shape[0]):
        if x[i] < -5/4 or x[i] > 5/4:
            arr[i] = 0
        elif x[i] >= -5/4 and x[i] <= -3/4:
            arr[i] = (1 + np.sin(2*pi*x[i]))/2
        
    return arr

def indicatrice_a(a,b):
    return lambda x: psi_a((2*x-b-a)/(b-a))

def indicatrice_b(a,b):
    return lambda x: psi_b((2*x-b-a)/(b-a))

def indicatrice_c(a,b):
    return lambda x: psi_c((2*x-b-a)/(b-a))


arr = np.linspace(-1, 1, npart + 1)
nn_indicatrici = [indicatrice_b(arr[i], arr[i+1]) for i in range(1,npart-1)]
nn_indicatrici.append(indicatrice_c(arr[-2], arr[-1]))
nn_indicatrici.insert(0, indicatrice_a(arr[0], arr[1]))

train_indicatrici = [nn_indicatrici[i](data.train_x) for i in range(npart)]
test_indicatrici = [nn_indicatrici[i](data.test_x) for i in range(npart)]

## Gestione punti partizione

In [ ]:
from abc import ABC, abstractmethod
from functools import wraps
import numpy as np

class Interval(Geometry):
    def __init__(self, l, r):
        super().__init__(1, (np.array([l]), np.array([r])), r - l)
        self.l, self.r = l, r

    def on_boundary(self, x, npart=1):
        if npart == 1:
            return np.any(isclose(x, [self.l, self.r]), axis=-1)
        else:
            pou_x = np.linspace(self.l, self.r, npart+1)
            return np.any(isclose(x, pou_x), axis=-1)
        
class BC(ABC):
    """Boundary condition base class."""
    def __init__(self, geom, on_boundary, component, npart=1):
        self.geom = geom
        self.npart = npart
        self.on_boundary = lambda x, on: np.array(
            [on_boundary(x[i], on[i], npart) for i in range(len(x))]
        )
        self.component = component

        self.boundary_normal = npfunc_range_autocache(
            utils.return_tensor(self.geom.boundary_normal)
        )

class DirichletBC(BC):
    """Dirichlet boundary conditions: y(x) = func(x)."""

    def __init__(self, geom, func, on_boundary, component=0, npart=1):
        super().__init__(geom, on_boundary, component, npart)
        self.func = npfunc_range_autocache(utils.return_tensor(func))

## Call Rete Neurale

In [ ]:
def call(self, inputs, training=False):
        res = 0
        
        for i in range(self.npart):
            y = inputs
            
            for f in self.denses[i]:
                y = f(y, training=training)
            
            if training == True:
                wei = self.train_indicatrici[i]
            else:
                wei = self.test_indicatrici[i]

            wei = tf.convert_to_tensor(wei)
            y = tf.math.multiply(y, wei)
            res += y

        return res